In [8]:
#import spacy
import pandas as pd

# Load English tokenizer, tagger, parser, NER, and word vectors
# nlp = spacy.load("en_core_web_sm")

# Load laptop data from CSV
laptops_df = pd.read_csv("laptop_price.csv")

# def preprocess_text(text):
#     """Preprocess the text using spaCy."""
#     doc = nlp(text)
#     # Extract price, RAM, and GPU information
#     price = None
#     ram = None
#     gpu = None
#     for token in doc:
#         if token.text.lower() == 'price':
#             price = next(doc[token.i + 1].like_num, None)
#         elif token.text.lower() == 'ram':
#             ram = next(doc[token.i + 1].like_num, None)
#         elif token.text.lower() == 'gpu':
#             gpu = next(doc[token.i + 1].like_num, None)
#     return price, ram, gpu

def recommend_laptop(price, ram, gpu):
    """Recommend a laptop based on price, RAM, and GPU."""
    # Filter laptops based on user input
    filtered_laptops = laptops_df
    if price:
        filtered_laptops = filtered_laptops[filtered_laptops['Price_euros'] <= price]
    if ram:
        filtered_laptops = filtered_laptops[filtered_laptops['Ram'] >= ram]
    if gpu:
        filtered_laptops = filtered_laptops[filtered_laptops['Gpu'] == gpu]

    # Sort laptops by price and return the top recommendation
    if not filtered_laptops.empty:
        return filtered_laptops.sort_values(by='Price_euros').iloc[0]
    else:
        return "No laptops match the specified criteria."

# Sample inputs
# sample_text = "I want a laptop with price around $1000, 16GB RAM, and Nvidia GPU."
# sample_price, sample_ram, sample_gpu = preprocess_text(sample_text)

sample_price = 900
sample_ram = "8GB"
sample_gpu = "Intel HD Graphics 6000"



# Recommend a laptop based on sample inputs
recommendation = recommend_laptop(sample_price, sample_ram, sample_gpu)
print(recommendation)


Unnamed: 0                               1
laptop_ID                                2
Company                              Apple
Product                        Macbook Air
TypeName                         Ultrabook
Inches                                13.3
ScreenResolution                  1440x900
Cpu                   Intel Core i5 1.8GHz
Ram                                    8GB
Memory                 128GB Flash Storage
Gpu                 Intel HD Graphics 6000
OpSys                                macOS
Weight                              1.34kg
Price_euros                         898.94
Name: 1, dtype: object
